In [1]:
from google.cloud import bigquery
from kaggle_secrets import UserSecretsClient
import pandas as pd

In [2]:
from google.cloud import bigquery
from kaggle_secrets import UserSecretsClient
import os

# Step 1: Load your GCP key from Kaggle Secrets
secrets = UserSecretsClient()
gcp_key = secrets.get_secret("gcp_key")

# Step 2: Save it locally
with open("gcp_key.json", "w") as f:
    f.write(gcp_key)

# Step 3: Set environment variable BEFORE creating client
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcp_key.json"

# Step 4: Create BigQuery client — no need to pass project manually
client = bigquery.Client()

# Step 5: Run your query
query = """
SELECT *
FROM `sunlit-hook-455618-n1.ed_data.sample_data`
LIMIT 10
"""
df = client.query(query).to_dataframe()
df.head()


Please ensure you specify a project id when creating the client in order to use your BigQuery account.


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/form

,subject_id,hadm_id,stay_id,intime,outtime,gender,race,arrival_transport,disposition,triage_temperature,...,vs_heartrate,vs_charttime,med_name,med_gsn,med_ndc,med_description,med_charttime,pyxis_name,pyxis_gsn,pyxis_charttime
0,10021487,28998349.0,38319705,2116-12-02 22:57:00+00:00,2116-12-03 01:02:00+00:00,M,WHITE,AMBULANCE,ADMITTED,NaN,...,NaN,NaT,None,<NA>,<NA>,None,NaT,PHENYLEPHRINE,5066.0,2116-12-03 00:23:00+00:00
1,10004235,24181354.0,38926302,2196-02-24 12:15:00+00:00,2196-02-24 17:07:00+00:00,M,MULTIPLE RACE/ETHNICITY,AMBULANCE,ADMITTED,NaN,...,NaN,NaT,None,<NA>,<NA>,None,NaT,EPINEPHRINE (FOR DILUTION),62006.0,2196-02-24 14:42:00+00:00
2,10021487,28998349.0,38319705,2116-12-02 22:57:00+00:00,2116-12-03 01:02:00+00:00,M,WHITE,AMBULANCE,ADMITTED,NaN,...,NaN,NaT,None,<NA>,<NA>,None,NaT,PROPOFOL,65888.0,2116-12-02 23:48:00+00:00
3,10022257,NaN,31940867,2167-06-07 16:23:00+00:00,2167-06-07 17:29:00+00:00,M,WHITE,UNKNOWN,ELOPED,96.9,...,NaN,NaT,None,<NA>,<NA>,None,NaT,None,NaN,NaT
4,10021487,28998349.0,38319705,2116-12-02 22:57:00+00:00,2116-12-03 01:02:00+00:00,M,WHITE,AMBULANCE,ADMITTED,NaN,...,NaN,NaT,None,<NA>,<NA>,None,NaT,PHENYLEPHRINE,5068.0,2116-12-03 00:24:00+00:00


In [3]:
from google.cloud import bigquery
import os
from kaggle_secrets import UserSecretsClient
import pandas as pd
import time

print("Authenticating with GCP and initializing BigQuery client...")
secrets = UserSecretsClient()
gcp_key = secrets.get_secret("gcp_key")
with open("gcp_key.json", "w") as f:
    f.write(gcp_key)
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "gcp_key.json"
client = bigquery.Client()
print(" Connected to BigQuery")

Authenticating with GCP and initializing BigQuery client...
Please ensure you specify a project id when creating the client in order to use your BigQuery account.
 Connected to BigQuery


In [16]:
query = """
WITH agg_diag AS (
  SELECT subject_id, stay_id,
         ARRAY_AGG(DISTINCT icd_title LIMIT 1)[OFFSET(0)] AS icd_title,
         ARRAY_AGG(DISTINCT icd_code LIMIT 1)[OFFSET(0)] AS icd_code
  FROM `sunlit-hook-455618-n1.ed_data.diagnosis`
  GROUP BY subject_id, stay_id
),
agg_med AS (
  SELECT subject_id, stay_id,
         ARRAY_AGG(DISTINCT name LIMIT 1)[OFFSET(0)] AS med_name,
         ARRAY_AGG(DISTINCT etcdescription LIMIT 1)[OFFSET(0)] AS etcdescription
  FROM `sunlit-hook-455618-n1.ed_data.medrecon`
  GROUP BY subject_id, stay_id
),
agg_pyxis AS (
  SELECT subject_id, stay_id,
         ARRAY_AGG(DISTINCT name LIMIT 1)[OFFSET(0)] AS pyxis_name,
         ARRAY_AGG(DISTINCT charttime LIMIT 1)[OFFSET(0)] AS pyxis_charttime
  FROM `sunlit-hook-455618-n1.ed_data.pyxis`
  GROUP BY subject_id, stay_id
),
agg_vitals AS (
  SELECT subject_id, stay_id,
         MAX(charttime) AS vs_charttime,
         MAX(heartrate) AS vs_heartrate
  FROM `sunlit-hook-455618-n1.ed_data.vitalsign`
  GROUP BY subject_id, stay_id
)

SELECT 
  t.subject_id, t.stay_id, t.chiefcomplaint, t.temperature, t.heartrate, t.resprate, t.o2sat, 
  t.sbp, t.dbp, t.pain, t.acuity,
  v.vs_charttime, v.vs_heartrate,
  d.icd_code, d.icd_title,
  m.med_name, m.etcdescription,
  p.pyxis_name, p.pyxis_charttime,
  e.hadm_id, e.intime, e.outtime, e.gender, e.race, e.arrival_transport
FROM `sunlit-hook-455618-n1.ed_data.triage` t
LEFT JOIN agg_diag d ON t.subject_id = d.subject_id AND t.stay_id = d.stay_id
LEFT JOIN agg_med m ON t.subject_id = m.subject_id AND t.stay_id = m.stay_id
LEFT JOIN agg_pyxis p ON t.subject_id = p.subject_id AND t.stay_id = p.stay_id
LEFT JOIN agg_vitals v ON t.subject_id = v.subject_id AND t.stay_id = v.stay_id
LEFT JOIN `sunlit-hook-455618-n1.ed_data.edstays` e ON t.subject_id = e.subject_id AND t.stay_id = e.stay_id
WHERE t.chiefcomplaint IS NOT NULL AND m.med_name IS NOT NULL AND d.icd_title IS NOT NULL
LIMIT 1000

"""

print(" Running BigQuery query...")
start = time.time()
merged = client.query(query).to_dataframe()
print(f" Query complete. Rows fetched: {len(merged)} | Time: {round(time.time() - start, 2)}s")

 Running BigQuery query...


/usr/local/lib/python3.11/dist-packages/google/cloud/bigquery/table.py:1727: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


 Query complete. Rows fetched: 1000 | Time: 10.74s


In [17]:
print(" Building retrieval context strings...")
def build_context(row):
    return f"""Medication: {row['med_name']}
Description: {row['etcdescription']}
Diagnosis: {row['icd_title']} ({row['icd_code']})
Chief Complaint: {row['chiefcomplaint']}"""

merged['context'] = merged.apply(build_context, axis=1)

 Building retrieval context strings...


In [18]:
!pip install faiss-cpu sentence-transformers transformers accelerate --quiet

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np

print("Loading embedding model...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("Encoding contexts for FAISS index...")
contexts = merged['context'].tolist()
embeddings = embedding_model.encode(contexts, show_progress_bar=True)
index = faiss.IndexFlatL2(embeddings.shape[1])
index.add(np.array(embeddings))
print("FAISS index built.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading embedding model...
Encoding contexts for FAISS index...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

FAISS index built.


In [19]:
merged.to_csv("merged_patient_data.csv", index=False)
print("📁 Saved merged dataset as 'merged_patient_data.csv'")
merged


📁 Saved merged dataset as 'merged_patient_data.csv'


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,subject_id,stay_id,chiefcomplaint,temperature,heartrate,resprate,o2sat,sbp,dbp,pain,...,etcdescription,pyxis_name,pyxis_charttime,hadm_id,intime,outtime,gender,race,arrival_transport,context
0,10000980,31236252,DYSPNEA,97.2,NaN,22.0,68.0,100.0,NaN,0,...,HYPERPARATHYROID TREATMENT AGENTS - VITAMIN D ...,ALBUTEROL 0.083% NEB SOLN 1NEB,2193-08-14 21:30:00+00:00,20897796.0,2193-08-14 21:25:00+00:00,2193-08-15 02:22:00+00:00,F,BLACK/AFRICAN AMERICAN,WALK IN,Medication: ATORVASTATIN\nDescription: HYPERPA...
1,10001667,33673933,"SLURRED SPEECH, TRANSFER",NaN,NaN,NaN,NaN,NaN,NaN,CRITICAL,...,FLUOROQUINOLONE ANTIBIOTICS,LABETALOL,2173-08-22 05:25:00+00:00,22672901.0,2173-08-22 01:40:00+00:00,2173-08-22 17:17:16+00:00,F,WHITE,AMBULANCE,Medication: ATENOLOL\nDescription: FLUOROQUINO...
2,10001884,31306678,HYPOXIA,NaN,NaN,NaN,79.0,NaN,NaN,UNABLE,...,GERIATRIC VITAMINS,METHYLPREDNISOLONE SODIUM SUCC,2130-10-19 13:50:00+00:00,28475784.0,2130-10-19 13:34:00+00:00,2130-10-19 18:50:00+00:00,F,BLACK/AFRICAN AMERICAN,WALK IN,Medication: ACETAMINOPHEN\nDescription: GERIAT...
3,10002428,33978784,RESP ARREST,NaN,NaN,NaN,NaN,NaN,NaN,NAN,...,HEPARINS,NOREPINEPHRINE,2156-05-11 11:55:00+00:00,23473524.0,2156-05-11 11:29:00+00:00,2156-05-11 14:49:34+00:00,F,WHITE,AMBULANCE,Medication: LEVOTHYROXINE\nDescription: HEPARI...
4,10002430,31293660,SOB,NaN,NaN,NaN,NaN,NaN,NaN,NAN,...,ANTIARRHYTHMIC - CLASS III,FUROSEMIDE,2125-09-28 18:05:00+00:00,24513842.0,2125-09-28 15:31:00+00:00,2125-09-28 22:22:00+00:00,M,WHITE,AMBULANCE,Medication: DOCUSATE SODIUM [COLACE]\nDescript...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,11572460,33444657,"CVA, TRANSFER",NaN,NaN,NaN,NaN,NaN,NaN,CRITICAL,...,BETA BLOCKERS CARDIAC SELECTIVE,PROPOFOL 1000MG/100ML 100ML VIAL,2177-09-17 13:19:00+00:00,25285710.0,2177-09-17 13:18:00+00:00,2177-09-17 19:00:00+00:00,F,WHITE,UNKNOWN,Medication: METOPROLOL TARTRATE\nDescription: ...
996,11575157,37259375,DYSPNEA,NaN,NaN,NaN,NaN,NaN,NaN,0,...,SALICYLATE ANALGESICS,IPRATROPIUM-ALBUTEROL NEB,2210-01-18 14:34:00+00:00,28645629.0,2210-01-18 14:29:00+00:00,2210-01-18 16:12:53+00:00,F,BLACK/AFRICAN AMERICAN,AMBULANCE,Medication: CALCIUM CITRATE-VITAMIN D3 [CITRAC...
997,11577197,35994371,FEVER,NaN,NaN,NaN,NaN,NaN,NaN,0,...,OPHTHALMIC-INTRAOCULAR PRESSURE REDUCING AGENT...,PIPERACILLI 4.5G/100ML 100ML BAG,2201-02-02 21:08:00+00:00,25655115.0,2201-02-02 19:03:00+00:00,2201-02-02 22:09:00+00:00,M,WHITE,AMBULANCE,Medication: MULTIVITAMIN\nDescription: OPHTHAL...
998,11579903,32252660,MVC,NaN,NaN,NaN,NaN,NaN,NaN,UTA,...,PLATELET AGGREGATION INHIBITORS - SALICYLATES,FENTANYL 100MCG/2ML 2ML AMP,2163-07-10 14:03:00+00:00,25509560.0,2163-07-10 13:52:00+00:00,2163-07-10 18:56:00+00:00,M,UNKNOWN,UNKNOWN,Medication: HYDROCODONE-ACETAMINOPHEN\nDescrip...


In [21]:
def safe(val):
    return str(val) if pd.notnull(val) else "N/A"

def build_context(row):
    return f"""Chief Complaint: {safe(row['chiefcomplaint'])}
Vitals → Temp: {safe(row['temperature'])}°F, HR: {safe(row['heartrate'])} bpm, RR: {safe(row['resprate'])}, BP: {safe(row['sbp'])}/{safe(row['dbp'])}, O2: {safe(row['o2sat'])}%
Pain: {safe(row['pain'])}, Acuity: {safe(row['acuity'])}
Diagnosis: {safe(row['icd_title'])} ({safe(row['icd_code'])})
Medication: {safe(row['med_name'])} | ETC: {safe(row['etcdescription'])}
Additional Med (Pyxis): {safe(row['pyxis_name'])} @ {safe(row['pyxis_charttime'])}
"""


In [33]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(" Model will run on:", device)


 Model will run on: cuda


In [40]:
!pip install faiss-cpu sentence-transformers transformers accelerate --quiet

from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import torch

print("Loading SentenceTransformer model...")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2", device="cuda" if torch.cuda.is_available() else "cpu")
print("Encoding all contexts...")
embeddings = embedding_model.encode(merged['context'].tolist(), show_progress_bar=True)
all_contexts = merged['context'].tolist()
embeddings = embedding_model.encode(all_contexts, show_progress_bar=True)
global_index = faiss.IndexFlatL2(embeddings.shape[1])
global_index.add(np.array(embeddings))
print(" FAISS index built with", global_index.ntotal, "entries.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Loading SentenceTransformer model...
Encoding all contexts...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Batches:   0%|          | 0/32 [00:00<?, ?it/s]

 FAISS index built with 1000 entries.


In [42]:
from typing import List
import re

from sklearn.metrics.pairwise import cosine_similarity

# Normalize and embed all unique medication names
med_names = merged['med_name'].dropna().unique()
med_cleaned = [re.sub(r'[^a-zA-Z0-9 ]', '', m.lower()) for m in med_names]
med_embeddings = embedding_model.encode(med_cleaned, convert_to_numpy=True)

def extract_med_from_query(query: str, top_k=5):
    query_tokens = re.findall(r'\b[a-z]+\b', query.lower())
    token_phrases = [" ".join(query_tokens[i:j]) for i in range(len(query_tokens)) for j in range(i+1, min(i+4, len(query_tokens))+1)]
    token_phrases = list(set(token_phrases))  # unique phrases

    phrase_embeddings = embedding_model.encode(token_phrases, convert_to_numpy=True)

    matches = set()
    for i, q_emb in enumerate(phrase_embeddings):
        sim_scores = cosine_similarity([q_emb], med_embeddings)[0]
        top_matches = np.argsort(sim_scores)[-top_k:]
        for idx in top_matches:
            if sim_scores[idx] > 0.6:  # Only keep confident matches
                matches.add((med_names[idx], sim_scores[idx]))

    matches = sorted(list(matches), key=lambda x: x[1], reverse=True)
    return [name for name, score in matches]



def retrieve_context(query: str, k: int = 3) -> List[str]:
    print(" Extracting keywords from query...")
    keywords = extract_med_from_query(query)
    print(f" Matched keywords: {keywords}")

    if keywords:
        pattern = "|".join([re.escape(k) for k in keywords])
        mask = merged['med_name'].str.contains(pattern, case=False, na=False)
        filtered_indices = merged[mask].index.to_list()
    else:
        filtered_indices = list(range(len(merged)))

    if not filtered_indices:
        print(" No matching rows found.")
        return []

    if len(filtered_indices) == len(merged):
        query_vec = embedding_model.encode([query])
        D, I = global_index.search(np.array(query_vec), k)
        return [all_contexts[i] for i in I[0]]

    filtered_contexts = [all_contexts[i] for i in filtered_indices]
    filtered_embeddings = np.array([embeddings[i] for i in filtered_indices])

    temp_index = faiss.IndexFlatL2(filtered_embeddings.shape[1])
    temp_index.add(filtered_embeddings)
    query_vec = embedding_model.encode([query])
    D, I = temp_index.search(np.array(query_vec), k)
    retrieved = [filtered_contexts[i] for i in I[0]]
    return list(dict.fromkeys(retrieved))  # removes duplicates while preserving order



from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("Loading MedAlpaca model with device_map='auto'...")
model_name = "medalpaca/medalpaca-7b"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
device_id = 0 if torch.cuda.is_available() else -1
medpipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

print("MedAlpaca model loaded.")

Batches:   0%|          | 0/14 [00:00<?, ?it/s]

Loading MedAlpaca model with device_map='auto'...


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Device set to use cpu


MedAlpaca model loaded.


In [43]:
query = "Why was vancomycin used?"
print(extract_med_from_query(query))
# ✅ ['VANCOMYCI 1000MG/200ML', 'some similar spelling', ...]


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

['VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN']


In [44]:
def generate_answer(query: str, retrieved: List[str]) -> str:
    prompt = f"""You are a clinical decision support assistant.

Step 1: Analyze the patient's symptoms, vitals, and diagnosis.

Step 2: Consider whether the prescribed medication fits the case.

Step 3: Justify the decision medically.

{chr(10).join(retrieved)}

Question: {query}
"""
    result = medpipe(prompt, max_new_tokens=300, do_sample=False)[0]['generated_text']
    return result.strip()

In [47]:
query = "What is reasoning behind prescribing vancomycin to a patient?"

# Step 1: Extract keywords
extract_med_from_query(query)  

# Step 2: Retrieve relevant cases based on keyword-aware filtering
retrieved = retrieve_context(query)


# Step 3: Use MedAlpaca to generate the clinical explanation
answer = generate_answer(query, retrieved)

# Step 4: Display the answer
print("\nAgent Response:\n", answer)


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Extracting keywords from query...


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

 Matched keywords: ['VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN', 'VANCOMYCIN']


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Agent Response:
 You are a clinical decision support assistant.

Step 1: Analyze the patient's symptoms, vitals, and diagnosis.

Step 2: Consider whether the prescribed medication fits the case.

Step 3: Justify the decision medically.

Chief Complaint: PICC LINE EVAL
Vitals → Temp: nan°F, HR: nan bpm, RR: nan, BP: nan/nan, O2: nan%
Pain: 5, Acuity: 3.0
Demographics: M, WHITE, Transport: WALK IN
Diagnosis: MECH COMPL OF INFUSION CATHETER, INITIAL ENCOUNTER (T82594A)
Medication: VANCOMYCIN | ETC: ANALGESIC OR ANTIPYRETIC NON-OPIOID
Additional Med (Pyxis): OXYCODONE (IMMEDIATE REL 5MG TAB @ 2187-05-11 15:00:00+00:00
Vitals Time: 2187-05-11 19:25:00+00:00, HR: 73.0
Chief Complaint: LINE EVAL
Vitals → Temp: 98°F, HR: nan bpm, RR: nan, BP: 108/90, O2: nan%
Pain: 6, Acuity: 3.0
Demographics: M, WHITE, Transport: WALK IN
Diagnosis: PRESSURE ULCER OF SACRAL REGION, UNSPECIFIED STAGE (L89159)
Medication: VANCOMYCIN | ETC: ANALGESIC OR ANTIPYRETIC NON-OPIOID
Additional Med (Pyxis): OXYCODONE (IM